In [1]:
import numpy as np
import datacube
from datacube.api import GridWorkflow
from madmex.wrappers import gwf_query
import fiona
import pprint
from madmex.io.vector_db import VectorDb, load_segmentation_from_dataset
from madmex.models import PredictObject
from madmex.models import PredictClassification, Tag, Region
from django.contrib.gis.geos import Polygon
from madmex.util import chunk
import json
from shapely.geometry import shape, mapping
from madmex.util.spatial import geometry_transform
from affine import Affine
from rasterio.features import rasterize
import rasterio

In [2]:
#name of prediction

In [3]:
name_predict = 's2_fiona_predict'
#predict_object_id = 1 #TODO: delete this and change it to a programatically way

In [29]:
pred_objects = PredictClassification.objects.filter(name=name_predict).prefetch_related('tag')

In [5]:
(pred_objects[0],pred_objects[1])

(<PredictClassification: PredictClassification object (20001)>,
 <PredictClassification: PredictClassification object (20002)>)

In [6]:
pred_objects.distinct('predict_object_id')

<QuerySet [<PredictClassification: PredictClassification object (30000)>, <PredictClassification: PredictClassification object (40000)>]>

In [7]:
pred_objects.distinct('model_id')

<QuerySet [<PredictClassification: PredictClassification object (30000)>, <PredictClassification: PredictClassification object (40000)>]>

In [8]:
pred_objects.values()

<QuerySet [{'id': 20001, 'tag_id': 17, 'predict_object_id': 1, 'model_id': 1, 'name': 's2_fiona_predict', 'confidence': 0.359813797313797, 'features_id': 0}, {'id': 20002, 'tag_id': 8, 'predict_object_id': 1, 'model_id': 1, 'name': 's2_fiona_predict', 'confidence': 0.448979656973709, 'features_id': 1}, {'id': 20003, 'tag_id': 16, 'predict_object_id': 1, 'model_id': 1, 'name': 's2_fiona_predict', 'confidence': 0.562443223443223, 'features_id': 2}, {'id': 20004, 'tag_id': 16, 'predict_object_id': 1, 'model_id': 1, 'name': 's2_fiona_predict', 'confidence': 0.532513992537313, 'features_id': 3}, {'id': 20005, 'tag_id': 8, 'predict_object_id': 1, 'model_id': 1, 'name': 's2_fiona_predict', 'confidence': 0.435706735864967, 'features_id': 4}, {'id': 20006, 'tag_id': 8, 'predict_object_id': 1, 'model_id': 1, 'name': 's2_fiona_predict', 'confidence': 0.425622072786252, 'features_id': 5}, {'id': 20007, 'tag_id': 8, 'predict_object_id': 1, 'model_id': 1, 'name': 's2_fiona_predict', 'confidence': 0.

In [9]:
qs_ids = pred_objects.distinct('predict_object_id')

In [10]:
qs_ids[0].predict_object_id

1

In [11]:
qs_ids[1].predict_object_id

2

In [12]:
len(pred_objects)

20000

In [13]:
seg = PredictObject.objects.filter(id=qs_ids[0].predict_object_id)

In [14]:
path = seg[0].path

In [15]:
#intersect with region Jalisco

In [16]:
region = 'Jalisco'

In [17]:
region_geom = Region.objects.get(name=region).the_geom

In [18]:
region_geojson = region_geom.geojson

In [19]:
geometry = json.loads(region_geojson)

In [20]:
proj4='+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'

In [21]:
geometry_proj = geometry_transform(geometry,proj4)

In [22]:
shape_Jalisco=shape(geometry_proj)

In [23]:
with fiona.open(path) as src:
    source_driver = src.driver
    source_crs = src.crs
    source_schema = src.schema
    pprint.pprint(src[0])

{'geometry': {'coordinates': [[(2427740.0, 1026160.0),
                               (2428250.0, 1026160.0),
                               (2428250.0, 1025650.0),
                               (2427740.0, 1025650.0),
                               (2427740.0, 1026160.0)]],
              'type': 'Polygon'},
 'id': '0',
 'properties': OrderedDict([('id', 0)]),
 'type': 'Feature'}


In [24]:
fc_schema = {'geometry': 'Polygon',
             'properties': {'code': 'int'}}

In [30]:
with fiona.open(path) as src:
    fc_pred = [(x[0]['geometry'], x[1].tag.numeric_code) for x in zip(src, pred_objects)]

In [31]:
fc_pred[0][0]

{'type': 'Polygon',
 'coordinates': [[(2427740.0, 1026160.0),
   (2428250.0, 1026160.0),
   (2428250.0, 1025650.0),
   (2427740.0, 1025650.0),
   (2427740.0, 1026160.0)]]}

In [32]:
fc_pred[0][1]

28

In [33]:
bbox = seg[0].the_geom

In [34]:
xmin, ymin, xmax, ymax = bbox.extent
resolution = 10.0

In [35]:
# Define output raster shape
nrows = int(((ymax - ymin) // resolution) + 1)
ncols = int(((xmax - xmin) // resolution) + 1)
shape_dim = (nrows, ncols)

In [36]:
arr = np.zeros((nrows, ncols), dtype=np.uint8)

In [37]:
aff = Affine(resolution, 0, xmin, 0, -resolution, ymax)

In [38]:
aff

Affine(10.0, 0.0, 2427740.0,
       0.0, -10.0, 1026160.0)

In [39]:
geometry_label_intersect = tuple()

In [40]:
geometry_label_intersect = [(mapping(shape(feat[0]).intersection(shape_Jalisco)),
                                     feat[1]) for feat in fc_pred if shape(feat[0]).intersects(shape_Jalisco)]

In [41]:
len(geometry_label_intersect)

10000

In [42]:
geometry_label_intersect[0:2]

[({'type': 'Polygon',
   'coordinates': (((2427740.0, 1026160.0),
     (2428250.0, 1026160.0),
     (2428250.0, 1025650.0),
     (2427740.0, 1025650.0),
     (2427740.0, 1026160.0)),)},
  28),
 ({'type': 'Polygon',
   'coordinates': (((2428250.0, 1026160.0),
     (2428750.0, 1026160.0),
     (2428750.0, 1025650.0),
     (2428250.0, 1025650.0),
     (2428250.0, 1026160.0)),)},
  11)]

In [43]:
rasterize(shapes=geometry_label_intersect, transform=aff, dtype=np.uint8, out=arr)

array([[28, 28, 28, ..., 27, 27,  0],
       [28, 28, 28, ..., 27, 27,  0],
       [28, 28, 28, ..., 27, 27,  0],
       ...,
       [28, 28, 28, ..., 28, 28,  0],
       [28, 28, 28, ..., 28, 28,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)

In [44]:
meta = {'driver': 'GTiff',
         'width': shape_dim[1],
         'height': shape_dim[0],
         'count': 1,
         'dtype': arr.dtype,
         'crs': proj4,
         'transform': aff,
         'compress': 'lzw',
        'nodata': 0}

In [45]:
filename='/shared_volume/tasks/2019/fiona_cloud_usage/classif_and_intersection_fiona.tif'

In [46]:
with rasterio.open(filename, 'w', **meta) as dst:
    dst.write(arr, 1)